# Evaluating an Unstable Sequence

As shown in [Parker, 1997](https://pdfs.semanticscholar.org/3b55/cfb2325bc081f113612204733c2a3bd650fe.pdf), some numerical sequences which can be computed analytically highlight the limitations of floating point arithmetic. One such sequence is the following:

$$u(k+1) = 111 - \frac{1130}{u(k)} + \frac{3000}{u(k) \times u(k-1)}$$

where $u(0) = 2$ and $u(1) = -4$. When evaluating $u(30)$, the fraction being resolved should return a value of approximately $6.00564868$, however, IEEE floating point standards are insufficient for solving this problem and the obbserved answer converges to $100$ instead. We show the value of MCA for identifying where this series diverges, and propose it as a method for evaluating the stability of arbitrary tools.

In [1]:
import os
import os.path as op
import numpy as np
from fractions import Fraction

import plotly.graph_objects as go

### Load log of outputs from floating point MCA simulation

We evaluated this sequence using floating point data types using the following function:

```python
import sys
eps = sys.float_info.epsilon

def u(K):
    if K == 0:
        return 2.0
    elif K == 1:
        return -4.0
    else:
        uminus2 = 2.0
        uminus1 = -4.0

    for k in range(2, K+1):
        val = 111 - (1130/(uminus1) + 3000/(uminus1 * uminus2)
        uminus2 = uminus1
        uminus1 = val
    return val
```

This function was evaluated 100 times using Monte Carlo Arithmetic, in which perturbations were introduced in the form of random bit-flips at the least significant bit in the represented data (bit $53$ of the mantissa). Each evaluation was saved in a text file, and is loaded here.

In [2]:
filename = './mca_instabilities.txt'
data = np.loadtxt(filename)

### Compute (better) estimate of the sequence

In this case, we can do a better job at estimating our sequence do to the fractional nature of the problem. Using Python's `Fraction` datatype, we get a much closer evaluation. This is used as a stand-in for the "True" sequence, but importantly is a) still not exact, and b) only possible in special cases, and is not a valid stand-in for floating point representations in many problems.

In [3]:
u = []
u += [Fraction(2)]
u += [Fraction(-4)]

for k in range(2, 30):
    u += [111 - 1130/u[-1] + 3000/(u[-1] * u[-2])]

### Plot the sequence

In [4]:
x = list(np.arange(data.shape[0]))
x_rev = x[::-1]

# Line 1
serr = np.std(data, axis=1) / np.sqrt(data.shape[1])
# serr = np.std(data, axis=1)
y1 = np.mean(data, axis=1)
y1_upper = list(y1 + serr)
y1_lower = list(y1 - serr)
y1_lower = y1_lower[::-1]

us = [float(_) for _ in u]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x+x_rev,
    y=y1_upper+y1_lower,
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name='Estimated U',
))
fig.add_trace(go.Scatter(
    x=x, y=y1,
    line_color='rgb(0,100,80)',
    name='Estimated U',
))
fig.add_trace(go.Scatter(
    x=x, y=us,
    line_color='rgb(240,80,0)',
    name='True U',
    line={"dash":"dash"}
))


fig.update_traces(mode='lines')
fig.update_layout(title='Evaluation of the sequence: u(k+1) = 111 - 1130/u(k) + 3000/(u(k) u(k-1))',
                  yaxis={"title": "U (log scale)", "type": "log", "tickvals":[6.00564868, 10, 100]},
                  xaxis={"title": "Sequence Value, k"},
                  width=1000,
                  height=500)
fig.show()

In [5]:
fig.write_image('./mca_sequence_example.pdf')
fig.write_image('./mca_sequence_example.png')